# Superconductivity Individual Analysis (New calibration template)

Use this template to carry out the analysis tasks for the experiment.  For reference, here are links to recommended Python resources: the [Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/) and the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) both by Jake VanderPlas.

We will be making use of both the [Uncertainties](https://pythonhosted.org/uncertainties/) and [LMFit](https://lmfit.github.io/lmfit-py/) packages in this notebook.

#### Updated to show revised analysis procedures

As of February 2024, use the steps outlined below if you 
* Have taken data with the LabVIEW based Chart Recorder application.
* Wish to use the ITS-90 coefficients to calculate temperature from the pressure data.

In [ ]:
# Run this cell with Shift-Enter, and wait until the asterisk changes to a number, i.e., [*] becomes [1]
import numpy as np
import scipy.constants as const
import uncertainties as unc
import uncertainties.unumpy as unp
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 9.0,6.5  # Roughly 9 cm wde by 6.5 cm high
mpl.rcParams['font.size'] = 11.0 # Use 11 point font

## Prior to working on this notebook:

1. Extract the transition points ($I$ through the 5 ohm sensing resistor in voltage units) versus pressure readings ($P$ in voltage units scaled to mmHg, called "torr") from the data scans. Make a CSV spreadsheet file with two columns: "P (torr)", "I (volts)"

2. Calculate the temperature in on of two ways:

    a. Use the "1958 He Scale of Temperatures” booklet, or the [online calculator from Quantum Design](https://qdusa.com/support/helium_vapor_pressure.html).
    
    b. Opt to create a function from the official polynomial fit described in the [International Temperature Scale of 1990 (ITS-90)](https://courses.washington.edu/phys431/superconductivity/ITS-1990_McGlashan_JChemThermo_1990.pdf), to convert the pressure readings to temperature $T$ (K) in the steps below.


### Then...
4. Import the CSV file of pressure/temperature versus transition points in volts across the current-sense resistor into the directory containing this notebook.

5. Create a calibration function to convert measured voltage across the current-sense resistor to magnetic field, using the magnet calibration curve in figure 5 of the (revised in 20204) instructions.

6. Apply the calibration function to obtain the transition points in terms of magnetic field in gauss.

7. Conduct a similar process if using the ITS-90 scale coefficients to obtain the temperature from the pressure reading.  (If you have already done this by hand, skip this step.)

8. Plot $H_c$ vs. $T$.

9. Fit the above using two different methods: (A) linearize the data, then fit to a line and obtain the slope and intercept, and (B) fit to the functional form of equation (1) in the instructions.  From both ways of fitting, find the critical temperature at zero field $T_c$ and the critical field at 0K $H_{c0}$.

10. Obtain the uncertainties in the results from the fits, and compare them to each other and to the values found in a handbook or text.


## Import data tables

Display them so that you know they are correct.

Import $P$ (volts,torr) and/or $T$ (kelvin) and current-sense voltage $I(\text{volts})$

In [ ]:
Hg_data = pd.read_csv('superconductivity_data_2024.csv')
Hg_data

Plot the raw data.  Always a good idea to make sure you actually have what you think you have!

In [ ]:
# You code this


## Make a magnet calibration function

Write a function that will convert the transition location in volts to field in gauss.  The Polynomial fit function is shown in the legend of the plot.

**Note:** The data are collected in the direction of **decreasing current** (normal to superconducting transition).

![Magnet calibration data and fit](magnet_cal_2024.png)

In [ ]:
## Make your calibration function.

## Then test it

coeffs = [16.9383, 80.8228, 0.4954, -0.1859]

def I2B(I, Cn=coeffs):
    '''
    Magnet calibration functionfor arbitrary polynomial length.
    Assumes that lowest index is constant term.
    '''
    # You code this
    
    return result


Overlay the conversion function on the calibration data.

## Apply the magnet calibration

Convert the transition locations to magnetic field.

In [ ]:
# Add a column o your data frame.
# See the examples for how one can do this.


## If using [ITS-90 coefficients](https://courses.washington.edu/phys431/superconductivity/ITS-1990_McGlashan_JChemThermo_1990.pdf), make a temperature function

Skip this step if you already have found the temperature from the online calculator or old look-up table.

Otherwise build a function that converts pressure to temperature from the following formula:

$$T/\text{K} = A_0 + \sum_{i=1}^{8} A_i\left[(\ln(p/\text{Pa})-B)/C\right]^i$$

NOTE: Pressure must be converted from torr to pascals!

The coefficients depend on the pressure.  Helium-4 undergoes a normal to superfluid transition at a particular point called the "lambda" point, which is at 31.81661 torr and 2.1768 K.  The vapor pressure curve varies slightly above and below this point.

| Coefficient |  $p$ $<$ 31.81661 torr | $p$ $\ge$ 31.81661 torr |
|    :---:    |        ---: |       ---: |
|    $A_0$    | 1.392408    | 3.146631   |
|    $A_1$    | 0.527153    | 1.357655   |
|    $A_2$    | 0.166756    | 0.413923   |
|    $A_3$    | 0.050988    | 0.091159   |
|    $A_4$    | 0.026514    | 0.016349   |
|    $A_5$    | 0.001975    | 0.001826   |
|    $A_6$    |-0.017976    |-0.004325   |
|    $A_7$    | 0.005409    |-0.004973   |
|    $A_8$    | 0.013259    | 0          |
|    $B$      | 5.6         | 10.3       |
|    $C$      | 2.9         | 1.9        |

Below, create your function and test it by feeding in a few values of pressure and checking the temerature output against the values obtained by the [online calculator from Quantum Design](https://qdusa.com/support/helium_vapor_pressure.html).

Then apply your function to your array of pressures to obtain the corresponding temperatures.  Don't forget!: your measurements may be in volts, not torr, and the coefficients assume that pressure is in pascals.

**ADVICE** 

Because of the decision that must be made which coefficients to use, it is tricky to make a fuction that will work on a whole NumPy array. But if you use the Uncertainties package version of the math functions, you can make your function propagate the uncertainty in temperature from an uncertainty in the pressure readings. 

SO: Make your function work on single values (possibly with uncertainty), then use the function in a loop to calculate the temperature values, and build an array with these.  (See the example code on GitHub for how to do this.) 

In [ ]:
# Build a DataFrame or pair of arrays from the coefficients

# Print them to double-check


In [ ]:
# Create the function.  The comments in the function body indicate steps to take.

def ITS90_p2T(pressure, p_lambda=31.81661, lowP_coeffs=L_coef, hiP_coeffs=H_coef):
    '''
    Convert pressure in torr to temperature in Kelvin from the ITS-90 coefficients and formula.
    This function is NOT NumPy safe, due to the boolean operation needed to choose the coefficients.
    But it can be used with uncertainty objects.
    '''
    # Convert lambda point to Pa
    
    # Convert pressure to Pa
    
    # Make a decision which set of coefficients to use, based on whether
    # the pressure is above or below the lambda point.
        
    # Calculate the argument x in the polynomial x = (ln(p) - B)/C
    # if you want to calculate with uncertainties, use unp.log(), othrwise np.log()
    
    # Build a loop to iterate over x
    
    # Return the calculated temperature
    return T

Test your pressure to temperature function.  Feed a few pressure values in, and check the results against the [online calculator from Quantum Design](https://qdusa.com/support/helium_vapor_pressure.html)

In [ ]:
# Recommended pressures: 760 torr should give typical BP of liquid helium at 4.2K.  
# Lowest pressure of about 3 torr should give near 1.5K.  Check a few points in between as well


### Convert $p$ to $T$

Apply your conversion to the pressure array.  You may need to make a loop to iterate, (unless you know how to deal with boolean operation in NumPy arrays)

In [ ]:
# Iterate over the pressure array, and build a new array with np.append()


In [ ]:
# Add this to the original dataFrame


## Plot the reduced data set

Plot the data (now in applied field versus temperature, or $H_c$ vs. $T$) so that it mirrors the graph shown in figure 1 of the instructions.  If you see peculiar data points, check for mistakes in transcription or other problems.

In [ ]:
# You code this


## Linearize the data

Rescale the data so that the relationship between the <i>x</i> and <i>y</i> values is linear.  Use equation (1) in the instructions as a guide:

$$H_c(T) = H_{c0}\left[1 - \left(\frac{T}{T_c}\right)^2\right]\quad\quad\quad\quad\quad(1)$$

Then plot the results to confirm.

In [ ]:
## Take the temperature values and square them
## Example from DataFrame: uT2 = Hg_data['T_90']*Hg_data['T_90']

# Then plot the results.  You should see a pretty good straight line.


## Fit the linearized data

Obtain the slope and intercept, plus uncertainty.

In [ ]:
# You code this



## Make a nice big, clear plot

Calculate the fit line from eval() and make a large plot, with a grid, data-points with error bars and the fit line on top of the data.  Make sure the fit line goes through both axes so that you can read the critical fiels and critical temperature-squared directly from the plot.

In [ ]:
## Make a big plot. Show the fit line going all the way 



## Obtain the critical field and temperature

Calculate $H_{c0}$ and $T_c$ from the fit results (with uncertainty).

In [ ]:
# You code this.


# print('Results from linearized data fit:')
# print('Hc0 = {:.2uP} gauss'.format(uHc0_1))
# print('Tc = {:.2uP} K'.format(uTc_1))

## Fit the "unlinearized" data

Use the LMFit `Model()` class to create a non-linear fit function in the form of equation (1).  Then use it to fit the unlinearized data of $H_c$ vs. $T$ directly.  In this version, the fit parameters should be the values of $H_{c0}$ vs. $T_c$.  

An example of using the `Model()` class to fit an arbitrary function is given in this notebook: [Example of LMFit Model Custom Function](Example%20LMFit%20Model%20with%20Arbitrary%20Function.ipynb) (The notebook should be in your working subdirectory).

In [ ]:
# Follow the example for a nonlinear fitting function in the example notebook.

# You will need to define the function correctly, and also provide staring 
# values for the critical tempeature and critical field.

## Define the fit function

## pull in the Model class and set up the fit

## First argument is the function name.  The others name the
##  adjustable parameters.

## Fit and plot


## Make another big, clear plot

Make a nice plot for this type of fit, similar to what you did above, showing the fit line intersecting both axes.

In [ ]:
## Make a big plot



## Compare the fit results

Compare the fit results from both procedures to each other: do they agree within their uncertainty?  Do the fit results agree with handbook or text values?  Print the all in a single table for easy comparison.

In [ ]:
# Example table
#
# print('Comparison of fit results:')
# print('   Tc (K)   |   Hc0 (G)   ')
# print('------------|-------------')
# print('{:.2uP} | {:.2uP}     (from linearized data)'.format(uTc_1,uHc0_1))
# print('{:.1uP} | {:.2uP}     (from nonlinear fit)'.format(uTc_2,uHc0_2))
# print('{:8.2f}    | {:.1f}         (handbook)'.format(4.15,410.0))